In [ ]:
a = np.empty((1,81))
a = np.full(81, np.nan)
geometry_array = a.reshape((9,9))
geometry_array

In [3]:
import numpy as np

# Two lists of data
ms_in = [1, 2, 4]
ns_in = [2, 4, 8]

ms_x = [2, 2, 2, 5, 9, 4]
ns = [4, 2, 2, 3, 6, 8]

# Find the indices (IDs) of pairs that exist in both lists
common_pairs = [[f, x] for f, (item1, item2) in enumerate(zip(ms_in, ns_in)) for x, (item3, item4) in enumerate(zip(ms_x, ns)) if (item1==item3 and item2==item4)]

# Print the common pairs
for pair in common_pairs:
    print(f"Common pair at indices {pair}: {ms_in[pair[0]], ms_x[pair[1]]}, {ns_in[pair[0]], ns[pair[1]]}")
    
a = np.array(common_pairs)
f_ids = a[:,0]
x_ids = a[:,1]

print(f_ids)
print(x_ids)

Common pair at indices [1, 0]: (2, 2), (4, 4)
Common pair at indices [2, 5]: (4, 4), (8, 8)
[1 2]
[0 5]


In [ ]:
import torch
import numpy as np
import torch.nn.functional as F


class X:
    def __init__(self, value, m, n):
        
        self.m = m # list of integers
        self.n = n # list of integers
        self.channels = value # list of tensors
                
    def set(self, value, m, n):
        self.m = m
        self.n = n
        self.channels = value
    
    def get(self):
        return self.channels, self.m, self.n
    
    def __str__(self):
        return 'X(channels: ' + str(self.channels.shape) +' at position: m=' + str(self.m) + ', n=' + str(self.n) + ')'
    __repr__ = __str__

    
    

class Filter(torch.nn.Module):
    def __init__(self, ms_in, ns_in, m_out, n_out, n_channels=1):
        super().__init__()
        
        self.ms_in = ms_in # list
        self.ns_in = ns_in # list
        self.m_out = m_out # single integer
        self.n_out = n_out # single integer
        self.weights = torch.autograd.Variable(torch.randn(1,n_channels,3,3))
        
    def forward(self, x_channels):
        
        #print(x.values())
    
        x_channels = torch.nn.functional.conv2d(x_channels, self.weights)
        
        #print(x.value)
        
        return x_channels
    
    def set(self, value, m_out, n_out):
        self.ms_in = None # list
        self.ns_in = None # list
        self.m_out = m_out # single integer
        self.n_out = n_out # single integer
        self.weights = value # weights in this filter
    
    def get(self):
        return self.m_out, self.n_out, self.weights
    
    def __str__(self):
        return 'DecentFilter(weights: ' + str(self.weights.shape) + ' at position: m_out=' + str(self.m_out) + ', n_out=' + str(self.n_out) + ')' + \
    '\n with inputs: ms_in= ' + ', '.join(str(m.item()) for m in self.ms_in) + ', ns_in= ' + ', '.join(str(n.item()) for n in self.ns_in) + ')'
    __repr__ = __str__
    
    
    
    
class Layer(torch.nn.Module):  
    def __init__(self, ms_in, ns_in, n_channels, n_filters):
        super().__init__()
        
        self.ms_in = ms_in
        self.ns_in = ns_in
                
        # use techniques from coo matrix
        self.geometry_array = np.full(81, np.nan)
        # plus 1 here cause of to_sparse array
        self.geometry_array[0:n_filters] = range(1,n_filters+1)
        np.random.shuffle(self.geometry_array)
        self.geometry_array = self.geometry_array.reshape((9,9), order='C')
        self.geometry_array = torch.tensor(self.geometry_array)
        self.geometry_array = self.geometry_array.to_sparse(sparse_dim=2).to("cuda")

        print(self.geometry_array)
        print(self.geometry_array.values())
        
        self.filter_list = torch.nn.ModuleList([])
        for i_filter in range(n_filters):
            # minus 1 here cause of to_sparse array
            index = (self.geometry_array.values()-1 == i_filter).nonzero(as_tuple=True)[0]
            m_out = self.geometry_array.indices()[0][index]
            n_out = self.geometry_array.indices()[1][index]
            self.filter_list.append(Filter(ms_in, ns_in, m_out, n_out, n_channels))
        
    def forward(self, x):
        
        print("the 1:", x)
        
        output_list = []
        for f in self.filter_list:
            output_list.append(f(x.channels))
        
        # needs to be for each channel??
        x.channels = torch.cat(output_list, dim=1)
        # mean = torch.mean(out, 0, keepdim=True)
        
        print("the 2:", x)
        
        return x #, pos
    
    def get_filter_positions(self):
        
        ms_out = []
        ns_out = []
        for f in self.filter_list:
            ms_out.append(f.m_out)
            ns_out.append(f.n_out)
        
        return ms_out, ns_out
    
    #def __str__(self):
    #    return 'Layer(filters: )' #  + str(self.weights.shape) + ' at position: m_out=' + str(self.m_out) + ', n_out=' + str(self.n_out) + ')'
    #__repr__ = __str__
    
    
    
    
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        dim = [1, 32, 48, 64, 10]
        dim = [1, 8, 16, 24, 10]
        assert not any(i > 81 for i in dim), "filters need to be less than 81"
        
        m0 = [torch.tensor(0)]
        n0 = [torch.tensor(0)]
        #self.layer0 = Layer(m,n)
        #m,n = layer0.get_positions()
        self.layer1 = Layer(m0, n0, dim[0], dim[1])
        m1,n1 = self.layer1.get_filter_positions()
        self.layer2 = Layer(m1, n1, dim[1], dim[2])
        m2,n2 = self.layer2.get_filter_positions()
        self.layer3 = Layer(m2, n2, dim[2], dim[-1])
        
        self.fc = torch.nn.Linear(dim[-1], dim[-1])
        
    def forward(self, x):
        # x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # from here we need to use the channels
        x = F.avg_pool2d(x.channels, kernel_size=x.channels.size()[2:])
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        return x
    
    
    def update(self):
        # measurement for updating
        
        # update layer by layer
    
        
        

In [ ]:
n = Net()

In [ ]:
n

In [1]:
tmp = torch.autograd.Variable(torch.randn(5, 1, 30, 30)) # batch x channel x width x height
# dense_input.shape

dense_input = X(tmp, 0, 0)

NameError: name 'torch' is not defined

In [ ]:
a = n(dense_input)

In [ ]:
torch.argmax(a, dim=1)
# one prediction for each image in the batch

In [ ]:
Filter([2, 3, 5], [8, 2, 4], 4, 1)

In [ ]:
print(a.value)

In [ ]:
import numpy as np
arr = np.full(81, 0)
print(arr)
arr[0:10] = range(0,10)
print(arr)
np.random.shuffle(arr)
arr = arr.reshape((9,9), order='C')
print(arr)



In [ ]:

dense_input = torch.tensor(arr)

co = dense_input.to_sparse(sparse_dim=2).to("cuda")
co

In [ ]:
index = (co.values() == 3).nonzero(as_tuple=True)[0]
co.indices()[0][index]
co.indices()[1][index]

In [ ]:
co.indices()[0][index]

In [ ]:
co.indices()[1][index]

In [ ]:
# trying with sparse ... doesn't work cause it gets applied on all dimensions or smthg ... I want a 9x9 matrix, not a 9x9x30x30 (grid, grid, img size, img size)

In [ ]:
import torch

class Filter(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
class Layer(torch.nn.Module):
    def __init__(self):
        super().__init__()
        a = 0
    def forward(self, x):
        
        #print(x.values())
        
        values = x.values()
        w_channels = 1
        weight = torch.autograd.Variable(torch.randn(1,w_channels,3,3))
        
        #print(x.shape)
        
        values_conv = torch.nn.functional.conv2d(values, weight)
        
        #print(x.shape)
        
        x = torch.sparse_coo_tensor(x.indices(), values_conv, x.size())
        
        return x
    
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer0 = Layer()
        self.layer1 = Layer()
        self.layer2 = Layer()
    def forward(self, x):
        # x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        return x
        
        

In [ ]:
n = Net()

In [ ]:
dense_input = torch.autograd.Variable(torch.randn(1,30,30)) # one image in batch
dense_input.shape

In [ ]:
coo_input = dense_input.to_sparse(sparse_dim=2)
coo_input

In [ ]:
# coo_input.values()

In [ ]:
a = n(coo_input)